In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import nn
import torch.nn.functional as f
from torch.autograd import Variable
import copy

from mmdet3d.models.temporal import ConvLSTM

In [3]:
def test(num_seqs, channels_img, size_image, max_epoch, model):
    input_image = torch.rand(num_seqs, 1, channels_img, size_image, size_image)
    # target_image = torch.rand(num_seqs, 1, channels_img, size_image, size_image)
    target_image = copy.deepcopy(input_image)
    # add some noise to the target image randomly
    for i in range(num_seqs):
        if i % 2 == 0:
            target_image[i] = input_image[i] + 0.1
        else:
            target_image[i] = input_image[i] - 0.1


    input_gru = Variable(input_image)
    target_gru = Variable(target_image)

    print("\n\n ==> Create a MSE criterion:")
    MSE_criterion = nn.MSELoss()

    h_next = None
    for e in range(max_epoch):
        for time in range(num_seqs):
            h_next = model(input_gru[time], h_next)
            print("h_next.shape", h_next.shape)
            print("target_gru.shape", target_gru.shape)
            print("target_gru[time].shape", target_gru[time].shape)
            err = MSE_criterion(h_next, target_gru[time])
            print("\n ... Error: " + str(err.cpu().detach().numpy()))

In [4]:
num_seqs = 3
hidden_size = 256
channels_img = 256
size_image = 200
max_epoch = 1
kernel_size = 3

In [5]:
# model = ConvGRUCell(channels_img, hidden_size, kernel_size)
# print(repr(model))
# test(num_seqs, channels_img, size_image, max_epoch, model)

In [19]:
conv_lstm = ConvLSTM(
    in_channels=256,
    hidden_channels=[256], # hparam
    kernel_size=[3,3],
    batch_first=True,
    bias=True,
    return_all_layers=False,
)

print(conv_lstm.__repr__())

ConvLSTM(
  (cell_list): ModuleList(
    (0): ConvLSTMCell(
      (conv): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
)


In [7]:
# B, C, H, W
# 1, 256, 200, 200
batch_size = 6
queue_length = 3
curr_feat = torch.rand(batch_size, 256, 200, 200)
print("curr_feat.shape", curr_feat.shape)
prev_feat = [torch.rand(batch_size, 256, 200, 200) for _ in range(queue_length - 1)]

# stack the previous features -> B, T, C, H, W
prev_feat = torch.stack(prev_feat, dim=1)
# stack the current features -> B, C, H, W -> B, 1, C, H, W
print("prev_feat.shape", prev_feat.shape)
curr_feat = curr_feat.unsqueeze(1)
print("curr_feat.shape", curr_feat.shape)
x = torch.cat((prev_feat, curr_feat), dim=1)
print("x.shape", x.shape)

curr_feat.shape torch.Size([6, 256, 200, 200])
prev_feat.shape torch.Size([6, 2, 256, 200, 200])
curr_feat.shape torch.Size([6, 1, 256, 200, 200])
x.shape torch.Size([6, 3, 256, 200, 200])


In [8]:
out = conv_lstm(x)

In [9]:
out.shape

torch.Size([6, 256, 200, 200])